In [1]:
import numpy as np
import pandas as pd

In [2]:
am1crop_emisioncrops_gfrac = pd.read_excel("D:/kerja/asisten riset/code v/step 3/GbGlct/AM1_prop_emission_crops_GFRAC_afterELKE_run3a.xlsx")
fao_area_harvested = pd.read_csv("D:/kerja/asisten riset/vol/milkunC/achaidir/FAO/faostat_area_harvested.csv")
ccode_iso = pd.read_excel("D:/kerja/asisten riset/vol/milkunC/achaidir/LUH2 2022/ISO-3166-Country-Code_REV.xlsx", sheet_name="Sheet3")

am1crop_emisioncrops_gfrac['country_name'] = am1crop_emisioncrops_gfrac['country_name'].replace("United Kingdom of Great Britain and Northern Irela", "United Kingdom of Great Britain and Northern Ireland")
am1crop_emisioncrops_gfrac = am1crop_emisioncrops_gfrac.rename(columns=lambda x: x.capitalize())
am1crop_emisioncrops_gfrac = am1crop_emisioncrops_gfrac.rename(columns={'Grass': 'grass'})

In [3]:
am1crop_emisioncrops_gfrac.rename(columns={"Country_name": "Country"}, inplace=True)
fao_area_harvested.rename(columns={"Area": "Country"}, inplace=True)

In [4]:
am1crop_emisioncrops_gfrac_merge = pd.merge(left=ccode_iso, right=am1crop_emisioncrops_gfrac, how="left", left_on="Country", right_on="Country")
fao_area_harvested_merge = pd.merge(left=ccode_iso, right=fao_area_harvested, how="left", left_on="Country", right_on="Country")

In [5]:
am1crop_emisioncrops_gfrac_merge_groubpy = am1crop_emisioncrops_gfrac_merge.groupby(['SubRegion', 'Time']).sum()
am1crop_emisioncrops_gfrac_merge_groubpy = am1crop_emisioncrops_gfrac_merge_groubpy.drop(columns=["Country", "Region"])
am1crop_emisioncrops_gfrac_merge_groubpy =  am1crop_emisioncrops_gfrac_merge_groubpy.reset_index()
am1crop_emisioncrops_gfrac_merge_groubpy['Time'] = am1crop_emisioncrops_gfrac_merge_groubpy['Time'].astype('int')
am1crop_emisioncrops_gfrac_merge_groubpy.head()

,SubRegion,Time,grass,Maize,Oil & palm fruit,Other non-food & luxury & spices,Other temperate cereals,Plant based fibres,Pulses,Rice,Soybeans,Sugar crops,Temperate oil crops,Temperate roots & tubers,Tropical cereals,Tropical oil crops,Tropical roots & tubers,Vegetables & fruits,Wheat
0,East Asia,1975,5.156366,6.436750,0.896661,5.763716,7.784545,4.247275,2.978531,7.641185,4.078861,2.075402,2.554195,1.359929,4.346996,1.296465,1.738305,3.762057,8.698617
1,East Asia,1980,5.388792,6.823091,2.652317,3.687769,6.437270,3.614562,3.079623,3.792050,4.784551,2.605308,3.295505,1.780218,3.142555,3.089690,1.851793,4.677397,9.130905
2,East Asia,1985,5.408749,8.887631,1.195713,4.471062,6.739863,4.746157,3.640141,1048.728053,5.476051,3.768184,4.104810,2.572052,3.080705,4.683175,2.212932,5.996095,9.425575
3,East Asia,1990,5.307354,869.617914,0.821562,5.255244,7.786621,5.003718,3.680075,1059.326838,6.274124,5.157689,4.783894,3.115556,2.245203,5.004009,3.405692,8.623916,9.661108
4,East Asia,1995,5.251788,918.159575,2.014033,6.038350,9.241941,4.340177,3.599330,451.074986,7.593428,5.907417,5.347759,3.886051,1.689292,4.202739,2.625300,12.849542,8.890708


In [6]:
fao_area_harvested_merge_groubpy = fao_area_harvested_merge.groupby(['SubRegion', 'Year', 'IMAGE Classification']).sum()
fao_area_harvested_merge_groubpy = fao_area_harvested_merge_groubpy.drop(columns=["Country", "Region", "Element"])
fao_area_harvested_merge_groubpy =  fao_area_harvested_merge_groubpy.reset_index()
fao_area_harvested_merge_groubpy['Year'] = fao_area_harvested_merge_groubpy['Year'].astype('int')
fao_area_harvested_merge_groubpy.head()

,SubRegion,Year,IMAGE Classification,True Value
0,East Asia,1970,Maize,43644.516700
1,East Asia,1970,Oil & palm fruit,5455.564587
2,East Asia,1970,Other non-food & luxury & spices,60011.210462
3,East Asia,1970,Other temperate cereals,49100.081287
4,East Asia,1970,Plant based fibres,43644.516700


if nilai fao data di area harvested == 0: continue;

In [7]:
fao_area_harvested_copy = fao_area_harvested_merge_groubpy.copy()
fao_area_harvested_copy['New Value'] = 0.0
fao_area_harvested_copy['New Value'].astype('float32')

for negara in np.unique(fao_area_harvested_merge_groubpy['SubRegion'].to_list()):
    for year in range(1975, 2025, 5):
        for ngfbfc in np.unique(fao_area_harvested_merge_groubpy['IMAGE Classification'].to_list()):
            try:
                am1crop_emisioncrops_gfrac_values = am1crop_emisioncrops_gfrac_merge_groubpy[(am1crop_emisioncrops_gfrac_merge_groubpy['SubRegion'] == negara) & (am1crop_emisioncrops_gfrac_merge_groubpy['Time'] == year)][ngfbfc].values
                fao_area_harvested_values = fao_area_harvested_merge_groubpy[(fao_area_harvested_merge_groubpy['Year'] == year) & (fao_area_harvested_merge_groubpy['SubRegion'] == negara) & (fao_area_harvested_merge_groubpy['IMAGE Classification'] == ngfbfc)]['True Value'].values
                fao_area_harvested_copy.loc[fao_area_harvested_merge_groubpy[(fao_area_harvested_merge_groubpy['SubRegion'] == negara) & (fao_area_harvested_merge_groubpy['Year'] == year) & (fao_area_harvested_merge_groubpy['IMAGE Classification'] == ngfbfc)].index, 'New Value'] = am1crop_emisioncrops_gfrac_values/fao_area_harvested_values
            except:
                print(negara)

In [9]:
fao_area_harvested_copy = fao_area_harvested_copy[fao_area_harvested_copy['Year'] > 1970]
fao_area_harvested_copy = fao_area_harvested_copy[["SubRegion", "Year", "IMAGE Classification", "New Value"]]
fao_area_harvested_copy

,SubRegion,Year,IMAGE Classification,New Value
16,East Asia,1975,Maize,0.000142
17,East Asia,1975,Oil & palm fruit,0.000159
18,East Asia,1975,Other non-food & luxury & spices,0.000093
19,East Asia,1975,Other temperate cereals,0.000153
20,East Asia,1975,Plant based fibres,0.000094
...,...,...,...,...
1535,sub-Saharan Africa,2020,Tropical cereals,0.000644
1536,sub-Saharan Africa,2020,Tropical oil crops,0.000010
1537,sub-Saharan Africa,2020,Tropical roots & tubers,0.000022
1538,sub-Saharan Africa,2020,Vegetables & fruits,0.000056


In [10]:
fao_area_harvested_copy.to_excel("D:/kerja/asisten riset/code v/step 3/GbGlct/AM1_emission_crops_footprint_subregion.xlsx", index=False)